In [18]:
#Build essentials
!pip install deap
!pip install gensim
#!pip install torch==1.7.1 torchvision==0.8.2 torchaudio==0.7.2
!pip install -U scikit-learn
!pip install -U skorch
!pip install datasets
!pip install -U nltk
!pip install torchsummary
!pip install pytorch-model-summary
!pip install matplotlib
!pip install seaborn
!pip install simpletransformers  
!pip install -U spacy

import pandas as pd
import os
import torch
import nltk

import pandas as pd
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn


from sklearn.model_selection import train_test_split

MAX_LEN_SENTENCE=0
from transformers import T5Tokenizer,AutoTokenizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in

In [24]:
from nltk.stem import WordNetLemmatizer 
import spacy
from spacy.lang.es.examples import sentences 
!python -m spacy download es_core_news_md
import es_core_news_md
from nltk import SnowballStemmer

def removing_stop_words(texts):
    # Removing stop words
    stop_words = set(stopwords.words('spanish'))
    for i, text in enumerate(texts):
        tokens = nltk.word_tokenize(text)
        sentence = [word for word in tokens if word not in stop_words]
        texts[i] = ' '.join(sentence)

def lemmatization(texts):
    nlp = es_core_news_md.load()
    for i, text in enumerate(texts):
        doc = nlp(text)
        sentence = [word.lemma_.lower() for word in doc]
        texts[i] = ' '.join(sentence)


def normalize(texts):
    nlp = es_core_news_md.load()
    for i, text in enumerate(texts):
        doc = nlp(text)
        words = [t.orth_ for t in doc if not t.is_punct | t.is_stop]
        lexical_tokens = [t.lower() for t in words if len(t) > 3 and t.isalpha()]   
        texts[i] = ' '.join(lexical_tokens) 


def prepare_input(texts):
    for i, text in enumerate(texts):
        texts[i] = "multilabel classification: {} </s>".format(text)


def tokenize(data):
    tokenizer = T5Tokenizer.from_pretrained("google/mt5-small")

    source_encoding = tokenizer(
        data['input_text'],
        max_length=200,
        padding='max_length',
        pad_to_max_length=True,
        truncation=True,
    )

    targets_polarity = [value[0] for value in data['target_text']]

    target_encoding_polarity = tokenizer(
        targets_polarity,
        max_length=2,
        pad_to_max_length=False,
        truncation=False,
    )

    targets_polarity = [int(value) for value in targets_polarity]

    encodings = {
        'source_ids': source_encoding['input_ids'],
        'target_ids': target_encoding_polarity['input_ids'],
        'attention_mask': source_encoding['attention_mask'],
        'labels': torch.tensor(targets_polarity, dtype=torch.long)
    }

    return encodings

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-11 00:53:13.234674: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42.3 MB 61.0 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')


In [25]:
    import nltk
    from nltk.corpus import wordnet as wn
    nltk.download('omw-1.4')
    nltk.download('wordnet')
    nltk.download('stopwords')
    nltk.download('punkt')
    file_train = "cuba_hotels_sentiment.xlsx"
    df = pd.read_excel(r'' + file_train, engine='openpyxl')
    print(df.iloc[0, 1])
    print(df.iloc[350, 2])
    print(df.shape)
    y = []
    X = []

    for i in range(df.shape[0]):
        whole_text = str(df.iloc[i, 0]) + ": " + str(df.iloc[i, 1])
        X.append(whole_text)
        y.append(str(df.iloc[i, 2]-1))

    X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size=0.30, random_state=142)
    stt_train = {'input_text': X_train, 'target_text': y_train}
    stt_dev = {'input_text': X_dev, 'target_text': y_dev}


    print("Number of instances for training: ")
    print(len(stt_train['input_text']))
    print("Number of instances for dev set: ")
    print(len(stt_dev['input_text']))
    
    print(stt_train['input_text'][0])
    normalize(stt_train['input_text'])
    print(stt_train['input_text'][0])
    normalize(stt_dev['input_text'])

    lemmatization(stt_train['input_text'])
    lemmatization(stt_dev['input_text'])

    prepare_input(stt_train['input_text'])
    prepare_input(stt_dev['input_text'])

    stt_train = tokenize(stt_train)
    stt_dev = tokenize(stt_dev)

    torch.save(stt_train, "dataset_train_chs_mt5_lemmatized")

    torch.save(stt_dev, "dataset_test_chs_mt5_lemmatized")

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Ha sido una experiencia maravillosa, estuvimos una Semana y fue la mejor semana, el personal es demasiado gentil, el desayuno es delicioso y hay mucha variedad y la vista al mar es muy hermosa ( aconsejo ver el atardecer) una de las mejores estadías que he tenido , volvería una y mil veces más
5
(17988, 4)
Number of instances for training: 
12591
Number of instances for dev set: 
5397
Mala atención: Pésima atención en la mesa buffet. No garantizan la comida de todos los clientes. Ante esto, indolencia de la gerencia. Las vistas son Expectaculares. La excursión muy bonita. Aún así no recomiendo el hotel. Gerencia mediocre
mala atención pésima atención mesa buffet garantizan comida clientes indolencia gerencia vistas expectaculares excursión bonita recomiendo hotel gerencia mediocre


/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5.py:226: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
